In [3]:
import torch
import tiktoken

In [4]:
with open ("Amontillado.txt", "r") as f:
    raw_text = f.read()

raw_text[:50]

'The thousand injuries of Fortunato I had borne as '

In [5]:
tokenizer = tiktoken.get_encoding("gpt2")


In [6]:
enc_text = tokenizer.encode(raw_text)

In [7]:
print(enc_text[:20])

[464, 7319, 6821, 286, 376, 1922, 5549, 314, 550, 28068, 355, 314, 1266, 714, 26, 475, 618, 339, 44716, 2402]


In [8]:
print(tokenizer.decode(enc_text[:20]))

The thousand injuries of Fortunato I had borne as I best could; but when he ventured upon


In [9]:
for i in range(1,10):
    print("INPUT:", tokenizer.decode(enc_text[:i]), "TARGET:", tokenizer.decode([enc_text[i]]))

INPUT: The TARGET:  thousand
INPUT: The thousand TARGET:  injuries
INPUT: The thousand injuries TARGET:  of
INPUT: The thousand injuries of TARGET:  F
INPUT: The thousand injuries of F TARGET: ortun
INPUT: The thousand injuries of Fortun TARGET: ato
INPUT: The thousand injuries of Fortunato TARGET:  I
INPUT: The thousand injuries of Fortunato I TARGET:  had
INPUT: The thousand injuries of Fortunato I had TARGET:  borne


## Data Set 

In [10]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]



## Dataloader

In [11]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [12]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


Inputs:
 tensor([[  464,  7319,  6821,   286],
        [  376,  1922,  5549,   314],
        [  550, 28068,   355,   314],
        [ 1266,   714,    26,   475],
        [  618,   339, 44716,  2402],
        [13277,    11,   314, 19982],
        [15827,    13,   921,    11],
        [  508,   523,   880,   760]])

Targets:
 tensor([[ 7319,  6821,   286,   376],
        [ 1922,  5549,   314,   550],
        [28068,   355,   314,  1266],
        [  714,    26,   475,   618],
        [  339, 44716,  2402, 13277],
        [   11,   314, 19982, 15827],
        [   13,   921,    11,   508],
        [  523,   880,   760,   262]])


In [13]:
type(inputs)

torch.Tensor

In [14]:
#Gives dimensions of the tensor
inputs.shape

torch.Size([8, 4])

In [15]:
#Look at the 2nd row
inputs[1]

tensor([ 376, 1922, 5549,  314])

In [16]:
# Look at the first column, first element of each row
inputs[:,0]

tensor([  464,   376,   550,  1266,   618, 13277, 15827,   508])

In [17]:
# Look at the ID in the 2nd row and 3rd column
inputs[1,2]

tensor(5549)

In [18]:
#Print tensor row by row, converting to list so tokenizer can decode
for row in inputs:
    print(tokenizer.decode(row.tolist()))

The thousand injuries of
 Fortunato I
 had borne as I
 best could; but
 when he ventured upon
 insult, I vowed
 revenge. You,
 who so well know


## Example of Generating Embedding Vectors

In [19]:
#illustrative example of how to generate the vectors of length 3 for the 6 word vocabular
vocab_size = 6
output_dim = 3

In [20]:
#nn is a sub-module for nueral networks in torch which contains Embedding
#Embedding created embedding vectors of a certain length for a certain vocab size
embedding = torch.nn.Embedding(vocab_size, output_dim)

In [21]:
print(embedding.weight)

Parameter containing:
tensor([[-2.2555,  0.7974,  1.6821],
        [-0.6824,  0.7153, -1.8959],
        [-0.2384, -0.0368,  0.6202],
        [ 0.0414,  0.4130,  0.5008],
        [-1.8873, -1.8476, -0.5438],
        [-0.7476, -0.4821,  0.9682]], requires_grad=True)


In [22]:
embedding.weight[0]

tensor([-2.2555,  0.7974,  1.6821], grad_fn=<SelectBackward0>)

In [23]:
#get rid of reqires_grad bit of text
embedding.weight.data

tensor([[-2.2555,  0.7974,  1.6821],
        [-0.6824,  0.7153, -1.8959],
        [-0.2384, -0.0368,  0.6202],
        [ 0.0414,  0.4130,  0.5008],
        [-1.8873, -1.8476, -0.5438],
        [-0.7476, -0.4821,  0.9682]])

In [24]:
# Or get rid of it using detach()
print(embedding.weight.detach())

tensor([[-2.2555,  0.7974,  1.6821],
        [-0.6824,  0.7153, -1.8959],
        [-0.2384, -0.0368,  0.6202],
        [ 0.0414,  0.4130,  0.5008],
        [-1.8873, -1.8476, -0.5438],
        [-0.7476, -0.4821,  0.9682]])


In [25]:
#What it would really look like with the correct sizes
#embedding = torch.nn.Embedding(tokenizer.n_vocab, 256)

In [26]:
A = embedding.weight.data
A

tensor([[-2.2555,  0.7974,  1.6821],
        [-0.6824,  0.7153, -1.8959],
        [-0.2384, -0.0368,  0.6202],
        [ 0.0414,  0.4130,  0.5008],
        [-1.8873, -1.8476, -0.5438],
        [-0.7476, -0.4821,  0.9682]])

In [27]:
#making tensors from scratch-- just put in the elements
x = torch.tensor([2.1,1.3])
y = torch.tensor([1.5,2.7])
print(x,y)

tensor([2.1000, 1.3000]) tensor([1.5000, 2.7000])


In [28]:
#computing the dot product if two tensors
torch.dot(x,y)

tensor(6.6600)

## Some More Practice Stuff on Attention Mechanisms

In [51]:
raw_text= "This is my dog Hazel"


In [52]:
tokenizer= tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text)
print (enc_text)

[1212, 318, 616, 3290, 42805]


In [53]:
vocab_size = 5
output_dim = 4
inputs = torch.nn.Embedding(vocab_size, output_dim)
print(inputs.weight)

Parameter containing:
tensor([[-0.3221,  0.8238, -0.9973,  0.7913],
        [ 0.2629, -0.0547,  0.7496,  0.0915],
        [ 0.2904, -0.3321, -1.1076, -0.0044],
        [ 0.7869,  0.5415, -0.0379,  0.4674],
        [-0.8365, -2.8908,  2.1387,  0.5873]], requires_grad=True)


In [54]:
#This gets rid of the grad=True thing
inputs = inputs.weight.data
inputs

tensor([[-0.3221,  0.8238, -0.9973,  0.7913],
        [ 0.2629, -0.0547,  0.7496,  0.0915],
        [ 0.2904, -0.3321, -1.1076, -0.0044],
        [ 0.7869,  0.5415, -0.0379,  0.4674],
        [-0.8365, -2.8908,  2.1387,  0.5873]])

In [55]:
inputs.shape

torch.Size([5, 4])

In [56]:
for row in inputs:
    print(row.tolist())

[-0.32212430238723755, 0.8237895369529724, -0.9973262548446655, 0.7912711501121521]
[0.262864351272583, -0.0546538382768631, 0.7495768070220947, 0.09148071706295013]
[0.29040059447288513, -0.3321409523487091, -1.1075810194015503, -0.0043955701403319836]
[0.7868971824645996, 0.5414729714393616, -0.03785805404186249, 0.4673885703086853]
[-0.8364709615707397, -2.8908259868621826, 2.138699769973755, 0.5872803926467896]


In [57]:
#Example to do dot product
x = torch.Tensor([1.1, 2.3])
y = torch.Tensor([3.4,-2.1])

In [58]:
#Calculate the dot product... annoying by hand
1.1*3.4 + 2.3*(-2.1)

-1.0899999999999999

In [59]:
#Built in dot product method!
torch.dot(x,y)

tensor(-1.0900)

In [69]:
#The important one
query = inputs[2]
print(query)

tensor([ 0.2904, -0.3321, -1.1076, -0.0044])


In [70]:
# gives the dot products of the 3rd embedding vector with all the other vectors (vector 3 x vector 1, vector 3 x vector 2 ...)
for i in range(len(inputs)):
    print(torch.dot(query, inputs[i]))

tensor(0.7340)
tensor(-0.7361)
tensor(1.4214)
tensor(0.0885)
tensor(-1.6541)


In [71]:
#Want the dot products (attention scores) in a tensor of their own
#torch.zeros creates a tensor of zeros so that you can fill them in

attention_scores_2 = torch.zeros(len(inputs))
for i in range(len(inputs)):
    attention_scores_2[i] = (torch.dot(query, inputs[i]))
print(attention_scores_2)

tensor([ 0.7340, -0.7361,  1.4214,  0.0885, -1.6541])


In [ ]:
# Normaling the attention scores using softmax function-- basically makes everything positive in a complicated way
#def softmax(x):
    #torch.exp(x) / torch.exp(x).sum()

In [72]:
#use the real torch function!
# Make sure you add the dimension
attention_scores_2 = torch.softmax(attention_scores_2, dim = 0)

In [73]:
attention_scores_2.sum()

tensor(1.0000)

In [ ]:
# Making the actual context vector ...
context_vector_2 = torch.zeros(query.shape)
for i in range(len(attention_scores_2)):
    context_vector_2 += attention_scores_2[i]*inputs[i]
context_vector_2

tensor([ 0.1699,  0.0442, -0.7435,  0.2875])